# **Novozymes-Enzyme-Stability-Prediction**

##### Develop models that can predict the ranking of protein stability (as measured by melting point, ```tm```) after single-point amino acid mutation and deletion. 

### **Import Libraries**

In [4]:
import Bio
from Bio import SeqIO
from io import StringIO
import biopandas
from biopandas.pdb import PandasPdb
pdb = PandasPdb()

import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

import numpy as np
import sklearn
from collections import Counter
from datetime import datetime
from zipfile import ZipFile
from glob import glob
import Levenshtein, warnings, requests, hashlib
import imageio, IPython, sklearn
import urllib, zipfile, pickle, random
import shutil, string, json, math, time, gzip
import ast, sys,io, os, gc, re

from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib
from matplotlib import animation, rc
rc('animation', html='jshtml')

import plotly, PIL, cv2
import plotly.io as pio

def seed_all(seed=7):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_all()

### **Helper Functions**

In [5]:
def get_mutation_info(_row, _wildtype="VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQ" \
                                      "RVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGT" \
                                      "NAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKAL" \
                                      "GSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"):
    terminology_map = {"replace":"substitution", "insert":"insertion", "delete":"deletion"}
    req_edits = Levenshtein.editops(_wildtype, _row["protein_sequence"])
    _row["n_edits"] = len(req_edits)
    
    if _row["n_edits"]==0:
        _row["edit_type"] = _row["edit_idx"] = _row["wildtype_aa"] = _row["mutant_aa"] = pd.NA
    else:
        _row["edit_type"] = terminology_map[req_edits[0][0]]
        _row["edit_idx"] = req_edits[0][1]
        _row["wildtype_aa"] = _wildtype[_row["edit_idx"]]
        _row["mutant_aa"] = _row["protein_sequence"][req_edits[0][2]] if _row["edit_type"]!="deletion" else pd.NA
    return _row

def revert_to_wildtype(protein_sequence, edit_type, edit_idx, wildtype_aa, mutant_aa):
    if pd.isna(edit_type):
        return protein_sequence
    elif edit_type!="insertion":
        new_wildtype_base = protein_sequence[:edit_idx]
        if edit_type=="deletion":
            new_wildtype=new_wildtype_base+wildtype_aa+protein_sequence[edit_idx:]
        else:
            new_wildtype=new_wildtype_base+wildtype_aa+protein_sequence[edit_idx+1:]
    else:
        new_wildtype=protein_sequence[:edit_idx]+wildtype_aa+protein_sequence[edit_idx:]
    return new_wildtype

def flatten_l_o_l(nested_list):
    """ Flatten a list of lists """
    return [item for sublist in nested_list for item in sublist]

def print_ln(symbol="-", line_len=110):
    print(symbol*line_len)
    
# Note mutation edit_idx is offset by 1 as many tools require it to be 1 indexed to 0 indexed.
def create_mutation_txt_file(_test_df, filename="/kaggle/working/AF70_mutations.txt", return_mutation_list=False, include_deletions=False):
    if return_mutation_list: mutation_list = []        
    with open(filename, 'w') as f:
        for _, _row in _test_df[["protein_sequence", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa"]].iterrows():
            if not include_deletions and (pd.isna(_row["edit_type"]) or _row["edit_type"]=="deletion"): continue
            f.write(f'{_row["wildtype_aa"]+str(_row["edit_idx"]+1)+(_row["mutant_aa"] if not pd.isna(_row["mutant_aa"]) else "")}\n')
            if return_mutation_list: mutation_list.append(f'{_row["wildtype_aa"]+str(_row["edit_idx"]+1)+(_row["mutant_aa"] if not pd.isna(_row["mutant_aa"]) else "")}')
    if return_mutation_list: return mutation_list
        
def create_wildtype_fasta_file(wildtype_sequence, filename="/kaggle/working/wildtype_af70.fasta"):
    with open(filename, 'w') as f: f.write(f">af70_wildtype\n{wildtype_sequence}")
        
def uniprot_id2seq(uniprot_id, _sleep=3):
    base_url = "http://www.uniprot.org/uniprot"
    full_url = os.path.join(base_url, str(uniprot_id)+".fasta")
    _r = requests.post(full_url)
    if _r.status_code!=200: print(_r.status_code)
    if _sleep!=-1: time.sleep(_sleep)
    return ''.join(_r.text.split("\n")[1:])

### **Loading Data**

In [6]:
# Define the path to the root data directory
DATA_DIR = "/home/linux/Workspace/mutating-doodle"
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
display(train_df)

# loading test dataframe from csv file
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
display(test_df)

# loading sample submission dataframe from csv file
ss_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
display(ss_df)

# load alphafold wildtype structure data from pdb file
pdb_df = pdb.read_pdb(os.path.join(DATA_DIR, "wildtype_structure_prediction_af2.pdb"))

# atom data
atom_df = pdb_df.df['ATOM']
display(atom_df)

# hetatm data
hetatm_df = pdb_df.df['HETATM']
display(hetatm_df)

# anisou data
anisou_df = pdb_df.df['ANISOU']
display(anisou_df)

# other data
others_df = pdb_df.df['OTHERS']
display(others_df)

# saving wildtype amino acid sequence
wildtype_aa = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"
print(wildtype_aa)

# define amino acid shortform dictionary mapping
aa_map = dict(Alanine=("Ala", "A"), Arginine=("Arg", "R"), Asparagine=("Asn", "N"), Aspartic_Acid=("Asp", "D"),
              Cysteine=("Cys", "C"), Glutamic_Acid=("Glu", "E"), Glutamine=("Gln", "Q"), Glycine=("Gly", "G"),
              Histidine=("His", "H"), Isoleucine=("Ile", "I"), Leucine=("Leu", "L"), Lysine=("Lys", "K"),
              Methionine=("Met", "M"), Phenylalanine=("Phe", "F"), Proline=("Pro", "P"), Serine=("Ser", "S"),
              Threonine=("Thr", "T"), Tryptophan=("Trp", "W"), Tyrosine=("Tyr", "Y"), Valine=("Val", "V"))
n_aa = len(aa_map)
aa_chars_ordered = sorted([v[1] for v in aa_map.values()])
aa_long2tri = {k:v[0] for k,v in aa_map.items()}
aa_long2char = {k:v[1] for k,v in aa_map.items()}
aa_tri2long = {v:k for k,v in aa_long2tri.items()}
aa_char2long = {v:k for k,v in aa_long2char.items()}
aa_char2int = {_aa:i for i, _aa in enumerate(aa_chars_ordered)}
aa_int2char = {v:k for k,v in aa_char2int.items()}

# Get data source map
ds_str2int = {k:i for i,k in enumerate(train_df["data_source"].unique())}
ds_int2str = {v:k for k,v in ds_str2int.items()}

for k,v in aa_map.items(): print(f"'{k}':\n\t3 LETTER ABBREVIATION --> '{v[0]}'\n\t1 LETTER ABBREVIATION --> '{v[1]}'\n")
    
print("\n\n... FOR FUN ... HERE IS THE ENTIRE WILDTYPE WITH FULL AMINO ACID NAMES (8 AA PER LINE) ...\n")
for i, _c in enumerate(wildtype_aa): print(f"'{aa_char2long[_c]}'", end=", ") if (i+1)%8!=0 else print(f"{aa_char2long[_c]}", end=",\n")

print("\n\n... ADD COLUMNS FOR PROTEIN SEQUENCE LENGTH AND INDIVIDUAL AMINO ACID COUNTS/FRACTIONS ...\n")
train_df["n_AA"] = train_df["protein_sequence"].apply(len)
test_df["n_AA"] = test_df["protein_sequence"].apply(len)
for _aa_char in aa_chars_ordered: 
    train_df[f"AA_{_aa_char}__count"] = train_df["protein_sequence"].apply(lambda x: x.count(_aa_char))
    train_df[f"AA_{_aa_char}__fraction"] = train_df[f"AA_{_aa_char}__count"]/train_df["n_AA"]
    test_df[f"AA_{_aa_char}__count"] = test_df["protein_sequence"].apply(lambda x: x.count(_aa_char))
    test_df[f"AA_{_aa_char}__fraction"] = test_df[f"AA_{_aa_char}__count"]/test_df["n_AA"]
    
print("\n... ADD COLUMNS FOR DATA SOURCE ENUMERATION ...\n")
train_df["data_source_enum"] = train_df['data_source'].map(ds_str2int)
test_df["data_source_enum"] = test_df['data_source'].map(ds_str2int)

print("\n... DO TEMPORARY pH FIX BY SWAPPING pH & tm IF pH>14 ...\n")
def tmp_ph_fix(_row):
    if _row["pH"]>14:
        print(f"\t--> pH WEIRDNESS EXISTS AT INDEX {_row.name}")
        _tmp = _row["pH"]
        _row["pH"] = _row["tm"]
        _row["tm"] = _tmp
        return _row
    else:
        return _row

print(f"\t--> DOES THE  STILL EXIST: {train_df['pH'].max()>14.0}")
train_df = train_df.progress_apply(tmp_ph_fix, axis=1)
test_df = test_df.progress_apply(tmp_ph_fix, axis=1)

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
...,...,...,...,...
2408,33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2409,33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2410,33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2411,33801,VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


,seq_id,tm
0,31390,0
1,31391,1
2,31392,2
3,31393,3
4,31394,4
...,...,...
2408,33798,2408
2409,33799,2409
2410,33800,2410
2411,33801,2411


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,blank_3,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,VAL,,A,1,,,34.064,-6.456,50.464,1.0,45.11,,,N,NaN,0
1,ATOM,2,,H,,VAL,,A,1,,,33.576,-6.009,51.228,1.0,45.11,,,H,NaN,1
2,ATOM,3,,H2,,VAL,,A,1,,,33.882,-7.449,50.477,1.0,45.11,,,H,NaN,2
3,ATOM,4,,H3,,VAL,,A,1,,,35.060,-6.323,50.566,1.0,45.11,,,H,NaN,3
4,ATOM,5,,CA,,VAL,,A,1,,,33.643,-5.877,49.162,1.0,45.11,,,C,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,ATOM,3313,,NZ,,LYS,,A,221,,,4.616,13.323,-4.301,1.0,93.80,,,N,NaN,3312
3313,ATOM,3314,,HZ1,,LYS,,A,221,,,5.270,12.565,-4.432,1.0,93.80,,,H,NaN,3313
3314,ATOM,3315,,HZ2,,LYS,,A,221,,,4.585,13.517,-3.310,1.0,93.80,,,H,NaN,3314
3315,ATOM,3316,,HZ3,,LYS,,A,221,,,4.965,14.143,-4.776,1.0,93.80,,,H,NaN,3315


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,blank_3,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx


...

,record_name,entry,line_idx
0,TER,3318 LYS A 221,3317
1,END,,3318


VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK
'Alanine':
	3 LETTER ABBREVIATION --> 'Ala'
	1 LETTER ABBREVIATION --> 'A'

'Arginine':
	3 LETTER ABBREVIATION --> 'Arg'
	1 LETTER ABBREVIATION --> 'R'

'Asparagine':
	3 LETTER ABBREVIATION --> 'Asn'
	1 LETTER ABBREVIATION --> 'N'

'Aspartic_Acid':
	3 LETTER ABBREVIATION --> 'Asp'
	1 LETTER ABBREVIATION --> 'D'

'Cysteine':
	3 LETTER ABBREVIATION --> 'Cys'
	1 LETTER ABBREVIATION --> 'C'

'Glutamic_Acid':
	3 LETTER ABBREVIATION --> 'Glu'
	1 LETTER ABBREVIATION --> 'E'

'Glutamine':
	3 LETTER ABBREVIATION --> 'Gln'
	1 LETTER ABBREVIATION --> 'Q'

'Glycine':
	3 LETTER ABBREVIATION --> 'Gly'
	1 LETTER ABBREVIATION --> 'G'

'Histidine':
	3 LETTER ABBREVIATION --> 'His'
	1 LETTER ABBREVIATION --> 'H'

'Isoleucine':
	3 LETTER ABBREVIATION --> 'Ile'
	1 LETTER ABBREVIATION -->

  0%|          | 0/31390 [00:00<?, ?it/s]

	--> pH WEIRDNESS EXISTS AT INDEX 973
	--> pH WEIRDNESS EXISTS AT INDEX 986
	--> pH WEIRDNESS EXISTS AT INDEX 988
	--> pH WEIRDNESS EXISTS AT INDEX 989
	--> pH WEIRDNESS EXISTS AT INDEX 1003
	--> pH WEIRDNESS EXISTS AT INDEX 1012
	--> pH WEIRDNESS EXISTS AT INDEX 1014
	--> pH WEIRDNESS EXISTS AT INDEX 1018
	--> pH WEIRDNESS EXISTS AT INDEX 1037
	--> pH WEIRDNESS EXISTS AT INDEX 1042
	--> pH WEIRDNESS EXISTS AT INDEX 1079
	--> pH WEIRDNESS EXISTS AT INDEX 1081
	--> pH WEIRDNESS EXISTS AT INDEX 1088
	--> pH WEIRDNESS EXISTS AT INDEX 1093
	--> pH WEIRDNESS EXISTS AT INDEX 1096
	--> pH WEIRDNESS EXISTS AT INDEX 1098
	--> pH WEIRDNESS EXISTS AT INDEX 1100
	--> pH WEIRDNESS EXISTS AT INDEX 1108
	--> pH WEIRDNESS EXISTS AT INDEX 1111
	--> pH WEIRDNESS EXISTS AT INDEX 1120
	--> pH WEIRDNESS EXISTS AT INDEX 1122
	--> pH WEIRDNESS EXISTS AT INDEX 1125
	--> pH WEIRDNESS EXISTS AT INDEX 13447
	--> pH WEIRDNESS EXISTS AT INDEX 13449
	--> pH WEIRDNESS EXISTS AT INDEX 14640
	--> pH WEIRDNESS EXISTS A

  0%|          | 0/2413 [00:00<?, ?it/s]

In [ ]:
# 1.1 – Define the path to the model output text file (I'm using mine but you can use the original)
DEEPDDG_PRED_TXT_PATH = "/kaggle/input/novoesp-deepddg-server-predictions-sub-only/wildtype_structure_prediction_af2.ddg.txt"

# 1.2 – Read the txt file into a pandas dataframe specifying a space as the delimiter
#       --> This introduces some weirdness with the number of columns... so we have to drop some and rename them
#       --> We also drop the #chain column as it contains no usable information (all rows are the same --> A)
#       --> We also rename the columns to be more user friendly (note ddg stands for ΔΔG)
deepddg_pred_df = pd.read_table(DEEPDDG_PRED_TXT_PATH, sep=" ").drop(columns=["#chain", "ddG", "is", "stable,", "is.1", "unstable)", "<0"])
deepddg_pred_df.columns = ["wildtype_aa", "residue_id", "mutant_aa", "ddg", "ddg_"]

# 1.3 – Coerce all the ddg values to be in the right column
deepddg_pred_df["ddg"] = deepddg_pred_df["ddg"].fillna(0.0)+deepddg_pred_df["ddg_"].fillna(0.0)
deepddg_pred_df = deepddg_pred_df.drop(columns=["ddg_"])

# 1.4 – Change edit location string name and change from 1-indexed to 0-indexed
deepddg_pred_df.loc[:,'location'] = deepddg_pred_df["residue_id"]-1
deepddg_pred_df = deepddg_pred_df.drop(columns=["residue_id"])

# 1.5 – Create a new column that contains the entire mutation as a string
#   --> This mutation string has the format   <wildtype_aa><location><mutant_aa>
deepddg_pred_df.loc[:,'mutant_string'] = deepddg_pred_df["wildtype_aa"]+deepddg_pred_df["location"].astype(str)+deepddg_pred_df["mutant_aa"]

# 1.6 – Display the newly created datafarme containing predictions (and describe float/int based columns)
display(deepddg_pred_df.describe().T)
display(deepddg_pred_df)

In [ ]:
# 2.1 – Adjust the residue column to be 0-indexed instead of 1-indexed
atom_df['residue_number'] -= 1

# 2.2 – Create a mapping from residue to b-factor 
#   --> b-factor is always the same for a particular residue
residue_to_bfactor_map = atom_df.groupby('residue_number').b_factor.first().to_dict()

In [ ]:
# 3.1 – Add mutation information about test set
#   --> Type of Mutation ['substitution'|'deletion'|'no_change'|'insertion'] (insertion never occurs)
#   --> Position of Mutation (Note this is now 0 indexed from the previous cell/section)
#   --> Wildtype Amino Acid (Single Letter Short Form)
#   --> Mutant Amino Acid (Single Letter Short Form – Will be NaN in 'deletion' mutations)
test_df = test_df.progress_apply(get_mutation_info, axis=1)

# 3.2 – Add b-factor to the test dataframe using previously created dictionary
test_df['b_factor'] = test_df["edit_idx"].map(residue_to_bfactor_map)

# 3.3 – Change edit_type from NaN to 'no_change'
#   --> this will allow the entire column to be a string as NaN is considered a float
test_df["edit_type"] = test_df["edit_type"].fillna("no_change")

# 3.4 – Change mutant_aa from NaN to '+' or '-' if edit_type is 'insertion' or 'deletion' respectively 
#   --> this will allow the entire column to be a string as NaN is considered a float
test_df.loc[test_df['edit_type']=='deletion', 'mutant_aa'] = '-'
test_df.loc[test_df['edit_type']=='insertion', 'mutant_aa'] = '+'

# 3.5 – Create a new column that contains the entire mutation as a string
#   --> This mutation string has the format   <wildtype_aa><edit_idx><mutant_aa>
test_df.loc[:, 'mutant_string'] =  test_df["wildtype_aa"]+test_df["edit_idx"].astype(str)+test_df["mutant_aa"]
test_df[["seq_id", "edit_type", "edit_idx", "wildtype_aa", "mutant_aa", "b_factor","mutant_string"]].head()

# 3.6 – Drop columns we don't need for this notebook
test_df = test_df[[_c for _c in test_df.columns if (("__" not in _c) and ("data_source" not in _c))]]

# 3.7 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 4.1 – Merge the two dataframes together 
test_df = test_df.merge(deepddg_pred_df[['ddg','mutant_string']], on='mutant_string', how='left')

# 4.2 – Fill in the missing ddg values with predetermined values
#   --> We set the default value for deletion to be equivalent to the bottom quartile value
#       of all substitutions... this is because it is more deleterious than simple substitutions
#   --> The default no_change value is simply 0.0 because this is the wildtype
#   --> What I would have thought is better: 
#            ----> DEFAULT__DELETION__DDG  = test_df[test_df["edit_type"]=="substitution"]["ddg"].quantile(q=0.25)
DEFAULT__DELETION__DDG  = -0.25
DEFAULT__NO_CHANGE__DDG = 0.0   # THIS IS DIFFERENT THAN THE ORIGINAL NOTEBOOK
test_df.loc[test_df['edit_type']=="deletion", 'ddg'] = DEFAULT__DELETION__DDG
test_df.loc[test_df['edit_type']=="no_change", 'ddg'] = DEFAULT__NO_CHANGE__DDG

# 4.3 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 5.1 – Define a function to return the substitution matrix (backwards and forwards)
from Bio.SubsMat import MatrixInfo
def get_sub_matrix(matrix_name="blosum100"):
    sub_matrix = getattr(MatrixInfo, matrix_name)
    sub_matrix.update({(k[1], k[0]):v for k,v in sub_matrix.items() if (k[1], k[0]) not in list(sub_matrix.keys())})
    return sub_matrix
sub_matrix = get_sub_matrix()

# 5.2 – Conduct matrix substitution
#   --> First we create a tuple that has the wildtype amino acid and the 
#       mutant amino acid to access the substitution matrix
#   --> Second we access the substitution matrix and replace with the respective score
#       and in cases where no respective score is found we mark it to be updated later
test_df["sub_matrix_tuple"] = test_df[["wildtype_aa", "mutant_aa"]].apply(tuple, axis=1)
test_df["sub_matrix_score"] = test_df["sub_matrix_tuple"].progress_apply(lambda _mutant_tuple: sub_matrix.get(_mutant_tuple, "tbd"))

# 5.3 – Fill in the missing data with default values for now
#   --> We set the default value for matrix sub to be equivalent to the bottom quartile value
#       of all substitutions... this is because it is more deleterious than simple substitutions (larger difference)
#   --> The default no_change value is 1 higher than the max score because a higher score means more similarity
#DEFAULT__DELETION__MATRIXSCORE  = test_df[test_df["edit_type"]=="substitution"]["sub_matrix_score"].quantile(q=0.25)
DEFAULT__DELETION__MATRIXSCORE = -10.0
#DEFAULT__NO_CHANGE__MATRIXSCORE = test_df[test_df["edit_type"]=="substitution"]["sub_matrix_score"].max()+1.0
DEFAULT__NO_CHANGE__MATRIXSCORE = 0.0
test_df.loc[test_df['edit_type']=="deletion", 'sub_matrix_score'] = DEFAULT__DELETION__MATRIXSCORE
test_df.loc[test_df['edit_type']=="no_change", 'sub_matrix_score'] = DEFAULT__NO_CHANGE__MATRIXSCORE
test_df["sub_matrix_score"] = test_df["sub_matrix_score"].astype(float)

# 5.4 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 6.1 – If the flag is set, reduce all positive matrix substitution scores to 0
CAP_SUB_MATRIX_SCORE = True
CAP_ABOVE_VAL, CAP_VAL = 0.0, 0.0
if CAP_SUB_MATRIX_SCORE:
    test_df.loc[test_df['sub_matrix_score'] > CAP_ABOVE_VAL, 'sub_matrix_score'] = CAP_VAL

# 6.2 – Normalize the matrix substitution score with adjusted sigmoid function
SIGMOID_ADJUSTMENT_CONSTANT = 3
def sigmoid_w_adjustment(x, adjustment_factor=3.0):
    return 1-(1/(1+np.exp(-x/adjustment_factor)))
test_df['sub_matrix_score_normalized'] = test_df['sub_matrix_score'].apply(lambda x: sigmoid_w_adjustment(x, SIGMOID_ADJUSTMENT_CONSTANT))
test_df['b_factor_matrix_score_adjusted'] = test_df['b_factor']*test_df['sub_matrix_score_normalized'] 

# 6.3 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
from scipy import stats

# 7.1 – Assign ranks to data, dealing with ties appropriately.
test_df['ddg_rank'] = stats.rankdata(test_df['ddg'])
test_df['sub_matrix_score_rank'] = stats.rankdata(test_df['sub_matrix_score'])
test_df['b_factor_rank'] = stats.rankdata(-test_df['b_factor'])
test_df['b_factor_matrix_score_adjusted_rank'] = stats.rankdata(-test_df['b_factor_matrix_score_adjusted'])

# 7.2 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 8.1 – Identify the columns to be combined
combo_cols = ['b_factor_rank', 'sub_matrix_score_rank', 'ddg_rank']

# 8.2 – Combine the columns by multiplying them together
test_df["combined_val"] = test_df[combo_cols].apply(np.prod, axis=1)

# 8.3 – Plot the newly created column to see the distribution of values
#       prior to any type of manipulation (i.e. raising to 1/3 power)
plt.figure(figsize=(14,6))
test_df["combined_val"].hist()
plt.title("Distribution of Values Prior To Being Raised to 1/3 Power")
plt.xlabel("b_factor_rank * sub_matrix_score_rank * ddg_rank", fontweight="bold")
plt.ylabel("frequency of occurence in dataset", fontweight="bold")
plt.show()

# 8.4 – 'Normalize' the combined value column by raising to a particular power
COMBO_NORM_PWR = 1/3
test_df["norm_combined_val"] = test_df["combined_val"]**COMBO_NORM_PWR

# 8.5 – Plot the newly created column to see the distribution of values
#       after manipulation (i.e. raising to 1/3 power)
plt.figure(figsize=(14,6))
test_df["norm_combined_val"].hist()
plt.title("Distribution of Combined Normalized Values After Being Raised to 1/3 Power")
plt.xlabel("(b_factor_rank * sub_matrix_score_rank * ddg_rank)^(1/3)", fontweight="bold")
plt.ylabel("frequency of occurence in dataset", fontweight="bold")
plt.show()

# 8.6 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 9.1 – Check that no NaN values slipped through
assert not test_df["norm_combined_val"].isna().any()

# 9.2 – Update the sample submission dataframe by creating a mapping and
#       applying it to the previously created sample_submission.csv
seqid_2_tmrank_mapping = test_df.groupby("seq_id")["norm_combined_val"].first().to_dict()
ss_df["tm"] = ss_df["seq_id"].apply(lambda x: seqid_2_tmrank_mapping[x])

# 9.3 – Save properly for submission
ss_df.to_csv("submission.csv", index=False)

# 9.4 – Display the sample submission dataframe and it's details
display(ss_df.describe().T)
display(ss_df)

In [ ]:
# 1.1 – Create substitution matrix for each position from DeMaSk data (apologies for the dump file names)
demask_sub_matrix = pd.read_csv("../input/demask-directional-substitution-matrix/inpsseq_mutation_ddg - Sheet2.csv")
demask_tuple_sub_map = {}
for _, _row in demask_sub_matrix.iterrows():
    demask_tuple_sub_map[(_row["pos"]-1, _row["WT"], _row["var"])] = _row["score"]
    demask_tuple_sub_map[(_row["pos"]-1, _row["var"], _row["WT"])] = _row["score"]

# 1.2 – Conduct matrix substitution
#   --> First we create a tuple that has the wildtype amino acid and the 
#       mutant amino acid to access the substitution matrix
#   --> Second we access the substitution matrix and replace with the respective score
#       and in cases where no respective score is found we mark it to be updated later
test_df["demask_sub_matrix_tuple"] = test_df[["edit_idx", "wildtype_aa", "mutant_aa"]].apply(tuple, axis=1)
test_df["demask_sub_matrix_score"] = test_df["demask_sub_matrix_tuple"].progress_apply(lambda _mutant_tuple: demask_tuple_sub_map.get(_mutant_tuple, "tbd"))

# 1.3 – Fill in the missing data with default values for now
#   --> We aim to make this as similar to the -10.0 that was previously used as possible
#       the most similar value considering the different scale would be --> -0.75
#   --> No change remains as 0.0
DEMASK_DEFAULT__DELETION__MATRIXSCORE = -0.75 # This is about 1.25 times larger than the minimum value (just like -10 in the original)
DEMASK_DEFAULT__NO_CHANGE__MATRIXSCORE = 0.0
test_df.loc[test_df['edit_type']=="deletion", 'demask_sub_matrix_score'] = DEMASK_DEFAULT__DELETION__MATRIXSCORE
test_df.loc[test_df['edit_type']=="no_change", 'demask_sub_matrix_score'] = DEMASK_DEFAULT__NO_CHANGE__MATRIXSCORE
test_df["demask_sub_matrix_score"] = test_df["demask_sub_matrix_score"].astype(float)

# 1.4 – Calculate the column rank similar to what we did previously
test_df['demask_sub_matrix_score_rank'] = stats.rankdata(test_df['demask_sub_matrix_score'])

# 1.5 – Plot the delta between the assigned rank values between the two approaches for raw sub_matrix_score
plt.figure(figsize=(18, 6))
(test_df["sub_matrix_score_rank"]-test_df["demask_sub_matrix_score_rank"]).abs().plot(kind="hist", bins=100)
plt.title("Disagreement Amount Between DeMaSk and BloSUM100", fontweight="bold")
plt.xlabel("Delta Change in Rank Between DeMaSk & BloSUM100", fontweight="bold")
plt.ylabel("Frequency of Occurence", fontweight="bold")
plt.grid(which="both")
plt.show()

# 1.6 – Identify the columns to be combined
combo_cols = ['b_factor_rank', 'demask_sub_matrix_score_rank', 'ddg_rank']

# 1.7 – Combine the columns by multiplying them together
test_df["demask_combined_val"] = test_df[combo_cols].apply(np.prod, axis=1)

# 1.7/ – To be fully added later... convert both outputs to rank
test_df["combined_val_rank"] = stats.rankdata(test_df['combined_val'])
test_df["deamsk_combined_val_rank"] = stats.rankdata(test_df['demask_combined_val'])

# 1.8 – Plot the newly created column to see the distribution of values
#       prior to any type of manipulation (i.e. raising to 1/3 power)
plt.figure(figsize=(14,6))
test_df["demask_combined_val"].hist()
plt.title("Distribution of Values Prior To Being Raised to 1/3 Power")
plt.xlabel("b_factor_rank * demask_sub_matrix_score_rank * ddg_rank", fontweight="bold")
plt.ylabel("frequency of occurence in dataset", fontweight="bold")
plt.show()

# 1.9 – 'Normalize' the combined value column by raising to a particular power (as we are using Spearman this does nothing)
COMBO_NORM_PWR = 1/3
test_df["demask_norm_combined_val"] = test_df["demask_combined_val"]**COMBO_NORM_PWR

# 1.10 – Plot the newly created column to see the distribution of values
#       after manipulation (i.e. raising to 1/3 power)
plt.figure(figsize=(14,6))
test_df["demask_norm_combined_val"].hist()
plt.title("Distribution of Combined Normalized Values After Being Raised to 1/3 Power")
plt.xlabel("(b_factor_rank * demask_sub_matrix_score_rank * ddg_rank)^(1/3)", fontweight="bold")
plt.ylabel("frequency of occurence in dataset", fontweight="bold")
plt.show()

# 1.11 – Plot the delta between the assigned rank values between the two approaches after combination and normalization
plt.figure(figsize=(18, 6))
(test_df["norm_combined_val"]-test_df["demask_norm_combined_val"]).abs().plot(kind="hist", bins=100)
plt.title("Disagreement Amount Between DeMaSk and BloSUM100", fontweight="bold")
plt.xlabel("Delta Change in Rank Between DeMaSk & BloSUM100", fontweight="bold")
plt.ylabel("Frequency of Occurence", fontweight="bold")
plt.grid(which="both")
plt.show()

# 1.12 – Create the submission file
demask_seqid_2_tmrank_mapping = test_df.groupby("seq_id")["demask_norm_combined_val"].first().to_dict()
ss_df["tm"] = ss_df["seq_id"].apply(lambda x: demask_seqid_2_tmrank_mapping[x])
ss_df.to_csv("demask_submission.csv", index=False)

# 1.13 – Display the sample submission dataframe and it's details
display(ss_df.describe().T)
display(ss_df)

# 1.14 – Display the updated test dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
# 1.11 – Plot the delta between the assigned rank values between the two approaches after combination and normalization
plt.figure(figsize=(18, 6))
(test_df["demask_combined_val"]-test_df["demask_norm_combined_val"]).abs().plot(kind="hist", bins=100)
plt.title("Disagreement Amount Between DeMaSk and BloSUM100", fontweight="bold")
plt.xlabel("Delta Change in Rank Between DeMaSk & BloSUM100", fontweight="bold")
plt.ylabel("Frequency of Occurence", fontweight="bold")
plt.grid(which="both")
plt.show()

In [ ]:
cols_to_ensemble = ["ddg_rank", "b_factor_rank", "sub_matrix_score_rank", "demask_sub_matrix_score_rank", ]
ss_df["tm"] = test_df[cols_to_ensemble].mean(axis=1)
ss_df.to_csv("submission.csv", index=False)

In [ ]:
test_df[test_df["seq_id"]==32559][[_c for _c in test_df.columns if 'rank' in _c]].T.rename(columns={1169:"wildtype_relative_ranking"})

In [ ]:
# 5.1 – Define a function to return the substitution matrix (backwards and forwards)
from Bio.SubsMat import MatrixInfo
def get_sub_matrix(matrix_name="blosum100"):
    sub_matrix = getattr(MatrixInfo, matrix_name)
    sub_matrix.update({(k[1], k[0]):v for k,v in sub_matrix.items() if (k[1], k[0]) not in list(sub_matrix.keys())})
    return sub_matrix
sub_matrix = get_sub_matrix()

# 5.2 – Conduct matrix substitution
#   --> First we create a tuple that has the wildtype amino acid and the 
#       mutant amino acid to access the substitution matrix
#   --> Second we access the substitution matrix and replace with the respective score
#       and in cases where no respective score is found we mark it to be updated later
test_df["sub_matrix_tuple_asym"] = test_df[["mutant_aa", "wildtype_aa"]].apply(tuple, axis=1)
test_df["sub_matrix_score_asym"] = test_df["sub_matrix_tuple_asym"].progress_apply(lambda _mutant_tuple: sub_matrix.get(_mutant_tuple, "tbd"))

# 5.3 – Fill in the missing data with default values for now
#   --> We set the default value for matrix sub to be equivalent to the bottom quartile value
#       of all substitutions... this is because it is more deleterious than simple substitutions (larger difference)
#   --> The default no_change value is 1 higher than the max score because a higher score means more similarity
#DEFAULT__DELETION__MATRIXSCORE  = test_df[test_df["edit_type"]=="substitution"]["sub_matrix_score"].quantile(q=0.25)
DEFAULT__DELETION__MATRIXSCORE = -10.0
#DEFAULT__NO_CHANGE__MATRIXSCORE = test_df[test_df["edit_type"]=="substitution"]["sub_matrix_score"].max()+1.0
DEFAULT__NO_CHANGE__MATRIXSCORE = 0.0
test_df.loc[test_df['edit_type']=="deletion", 'sub_matrix_score_asym'] = DEFAULT__DELETION__MATRIXSCORE
test_df.loc[test_df['edit_type']=="no_change", 'sub_matrix_score_asym'] = DEFAULT__NO_CHANGE__MATRIXSCORE
test_df["sub_matrix_score_asym"] = test_df["sub_matrix_score_asym"].astype(float)
test_df["sub_matrix_score_asym_rank"] = stats.rankdata(test_df["sub_matrix_score_asym"])

# 5.4 – Display the updated dataframe (and describe float/int based columns)
display(test_df.describe().T)
display(test_df)

In [ ]:
cols_to_ensemble = ["ddg_rank", "ddg_rank", "ddg_rank", "b_factor_rank", "b_factor_rank", "b_factor_rank", "sub_matrix_score_rank", "sub_matrix_score_rank", "sub_matrix_score_asym_rank", "demask_sub_matrix_score_rank", "demask_sub_matrix_score_rank", "demask_sub_matrix_score_rank"]
ss_df["tm"] = test_df[cols_to_ensemble].mean(axis=1)
ss_df.to_csv("submission.csv", index=False)